1. Get Fed Statements (text documents)
   ↓
2. Extract NLP Features (sentiment, tone, complexity, etc.)
   ↓
3. Calculate Market Reactions (% change in SPY, gold, etc.)
   ↓
4. Build ML Model (predict market reaction from NLP features)
   ↓
5. Analyze Results (which linguistic features matter most?)

# The Big Picture

You're basically asking: "Does the way the Fed talks predict how markets move?"
Your hypothesis: It's not just what the Fed decides (rate hike/cut/hold), but how they say it that matters. A hawkish tone might spook markets even with no rate change, while a dovey, reassuring tone might calm markets even with a rate hike.


# Feature Extraction Pipeline

## A. Sentiment Analysis
Goal: Measure positive/negative/neutral tone
Models to use:
- FinBERT (finance-specific BERT model)
    - Outputs: positive, negative, neutral scores (0-1)
    - Why: Trained on financial text, understands market language

- VADER (rule-based sentiment)
    - Outputs: compound score (-1 to +1)
    - Why: Fast, good baseline, handles intensity modifiers

Features extracted:
sentiment_finbert_pos, sentiment_finbert_neg, sentiment_finbert_neutral, sentiment_vader_compound

## B. Uncertainty/Risk Detection
Goal: Measure how uncertain or risky the language is
Model to use:
- Loughran-McDonald Financial Dictionary (word lists)
    - Counts words like "uncertain", "risk", "may", "could"
    - Outputs: uncertainty score, risk score (count or percentage)

Features extracted:
uncertainty_count (# of uncertainty words)
risk_count (# of risk words)


## C. Readability/Complexity
Goal: Measure how complex or "foggy" the language is
Metrics to use:
- Flesch Reading Ease (0-100, higher = easier)
- Flesch Kincaid Score (0-15+, readability test that measures the complexity of a text on a U.S. school grade level: lower = easier)
- Gunning Fog Index (years of education needed)
- Coleman-Liau Index (0-14+, Readability formula that estimates the U.S. grade level required to understand a text by using the number of letters per 100 words and the number of sentences per 100 words: lower=easier)
- Average sentence length
- Lexical diversity (unique words / total words)

Features extracted:
flesch_score
fog_index
avg_sentence_length
lexical_diversity
Why it matters: More complex statements might signal uncertainty or intentional obfuscation.





# **Feature Extraction Pipeline Proposal**

Input: Fed statement text
  ↓
Step 1: Preprocessing
  - Tokenization, lowercasing, remove special chars
  ↓
Step 2: Sentiment Analysis
  - Run FinBERT → get pos/neg/neutral scores
  - Run VADER → get compound score
  ↓
Step 3: Uncertainty/Risk Detection
  - Count Loughran-McDonald dictionary words
  ↓
Step 4: Readability Metrics
  - Calculate Flesch, Fog Index, etc.
  ↓
Output: n-dimensional feature vector per Fed date


**Output(Y): Market Reaction - Categorical  (Classification)**
- `market_reaction` = "positive", "neutral", or "negative"
- Example: If SPY goes up >1%, label it "positive"

**Each row = one Fed announcement**
**Columns:**
- **Features (X):** finbert_pos, finbert_neg, vader, uncertainty, etc.
- **Target (Y):** SPY_change, gold_change, etc.

## **Dimensionality Reduction (PCA)**

### **Why might you need it?**

If you have **too many features** (say 50+), some might be:
- **Redundant:** finbert_positive and vader_compound might measure similar things
- **Noisy:** Some features might not matter at all
- **Correlated:** Flesch and Fog Index are often highly correlated

Dimensionality reduction **compresses** your features while keeping the important information.

### **How PCA works (simple explanation):**

Imagine you have 10 features. PCA finds **new axes** (principal components) that capture the most variation in your data.

**Before PCA:**
```
X = [finbert_pos, finbert_neg, vader, uncertainty, ...]  # 10 features
```

**After PCA:**
```
X_reduced = [PC1, PC2, PC3, PC4, PC5]  # 5 features